<a href="https://colab.research.google.com/github/financieras/gym/blob/main/frozen_lake/map_frozen_lake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mapas para el juego Frozen Lake
Este juego lo resolveremos con aprendizaje automático usando el algoritmo Q-Learning y crearemos la Q-table.

## Determinar si un mapa es válido

In [1]:
def validar_mapa(mapa):
    """
    Valida un mapa de Frozen Lake según las especificaciones actualizadas.

    Args:
        mapa: Lista de strings que representan el mapa (nxn con n ≥ 2).
              S: Punto inicial, ·: Superficie helada, H: Hoyo, G: Meta.

    Returns:
        tuple: (bool, str) donde:
               - bool: True si el mapa es válido, False si no.
               - str: Mensaje descriptivo del resultado o error.
    """
    # Validación básica de tipo y estructura
    if not isinstance(mapa, list) or not mapa:
        return False, "El mapa debe ser una lista no vacía"

    n = len(mapa)
    if n < 2:
        return False, "El mapa debe tener al menos 2 filas"

    # Verificación de estructura cuadrada (nxn)
    if not all(isinstance(fila, str) and len(fila) == n for fila in mapa):
        return False, f"El mapa debe ser cuadrado (nxn). Todas las filas deben ser strings de longitud {n}"

    # Conteo de caracteres especiales y validación
    caracteres_validos = {'S', '·', 'H', 'G'}
    caracteres_presentes = set()
    contador_S = 0
    contador_G = 0

    for fila in mapa:
        for c in fila:
            caracteres_presentes.add(c)
            if c == 'S':
                contador_S += 1
            elif c == 'G':
                contador_G += 1

    # Validación de caracteres
    caracteres_invalidos = caracteres_presentes - caracteres_validos
    if caracteres_invalidos:
        return False, f"Caracteres inválidos encontrados: {', '.join(sorted(caracteres_invalidos))}"

    # Validación de S y G
    if contador_S != 1:
        return False, f"Debe haber exactamente un 'S' (inicio). Encontrados: {contador_S}"

    if contador_G != 1:
        return False, f"Debe haber exactamente un 'G' (meta). Encontrados: {contador_G}"

    return True, f"Mapa {n}x{n} válido"

In [2]:
# Mapa válido
mapa_valido = [
    "S···",
    "·H·H",
    "··H·",
    "H··G"
]
print(validar_mapa(mapa_valido))  # (True, "Mapa 4x4 válido")

# Mapa con caracteres inválidos
mapa_invalido = [
    "S·X·",
    "·H·H",
    "··H·",
    "H··G"
]
print(validar_mapa(mapa_invalido))  # (False, "Caracteres inválidos encontrados: X")

# Caso inválido (fila 2 más corta)
mapa2 = [
    "S···",
    "·H·H",
    "···",  # ¡Falta un carácter!
    "H··G"
]

print(validar_mapa(mapa2))  # (False, 'El mapa debe ser cuadrado (nxn). Todas las filas deben ser strings de longitud 4')

(True, 'Mapa 4x4 válido')
(False, 'Caracteres inválidos encontrados: X')
(False, 'El mapa debe ser cuadrado (nxn). Todas las filas deben ser strings de longitud 4')


## Calcular cuántos caminos minimos existen y el número de pasos necesarios

In [18]:
from collections import deque

def contar_caminos_minimos(mapa):
    """
    Encuentra el número de caminos mínimos y su longitud desde S hasta G.

    Args:
        mapa: Lista de strings que representan el mapa (debe ser válido).

    Returns:
        tuple: (num_caminos, pasos) donde:
               - num_caminos: Número de caminos mínimos (0 si no hay camino)
               - pasos: Número de pasos del camino mínimo (-1 si no hay camino)
    """
    # Validación inicial del mapa (omitiendo mensajes para brevedad)
    valido, _ = validar_mapa(mapa)
    if not valido:
        return 0, -1

    # Encontrar posiciones de S y G
    S_pos = next((i, j) for i, row in enumerate(mapa)
                   for j, c in enumerate(row) if c == 'S')
    G_pos = next((i, j) for i, row in enumerate(mapa)
                   for j, c in enumerate(row) if c == 'G')

    # Inicialización
    filas, cols = len(mapa), len(mapa[0])
    direcciones = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    distancia = [[-1] * cols for _ in range(filas)]
    contador = [[0] * cols for _ in range(filas)]

    # BFS
    q = deque([S_pos])
    distancia[S_pos[0]][S_pos[1]] = 0
    contador[S_pos[0]][S_pos[1]] = 1

    while q:
        i, j = q.popleft()

        for di, dj in direcciones:
            ni, nj = i + di, j + dj
            if 0 <= ni < filas and 0 <= nj < cols and mapa[ni][nj] != 'H':
                if distancia[ni][nj] == -1:  # Primera visita
                    distancia[ni][nj] = distancia[i][j] + 1
                    contador[ni][nj] = contador[i][j]
                    q.append((ni, nj))
                elif distancia[ni][nj] == distancia[i][j] + 1:  # Ruta alternativa mínima
                    contador[ni][nj] += contador[i][j]

    pasos = distancia[G_pos[0]][G_pos[1]]
    num_caminos = contador[G_pos[0]][G_pos[1]] if pasos != -1 else 0

    return num_caminos, pasos

In [21]:
mapa_4x4 = [
    "S···",
    "·H·H",
    "···H",
    "H··G"
]

valido, mensaje = validar_mapa(mapa_4x4)
print(f"¿Mapa válido? {valido}, Mensaje: {mensaje}")

num_caminos, pasos = contar_caminos_minimos(mapa_4x4)
print(f"Caminos mínimos: {num_caminos}, Pasos requeridos: {pasos}")

¿Mapa válido? True, Mensaje: Mapa 4x4 válido
Número de caminos mínimos: 3 resuelto en 6 pasos.


In [23]:
mapa = [
                "S·······",
                "·HHH·HH·",
                "·H·H····",
                "·H···H·H",
                "···H····",
                "·HH···H·",
                "·H··H·H·",
                "·······G"
            ]

valido, mensaje = validar_mapa(mapa)
print(f"¿Mapa válido? {valido}, Mensaje: {mensaje}")

num_caminos, pasos = contar_caminos_minimos(mapa)
print(f"Caminos mínimos: {num_caminos}, Pasos requeridos: {pasos}")

¿Mapa válido? True, Mensaje: Mapa 8x8 válido
Caminos mínimos: 5, Pasos requeridos: 14
